<p align="left">
<img width="50%" src="https://drive.google.com/uc?export=view&id=10y3NKbbk7yt7cZDMszMt04g6NquTEa4p" alt="Carbon Logo" />
</p>


# Carbon Simulator - Litepaper Examples

In this notebook we demonstrate how to set up specific scenarios on Carbon:

* **Limit Order (sell)** - Here we set up a strategy where the AMM sells ETH for USDC at 2000 (USDC per ETH).
* **Limit Order (buy)** - Bob creates a strategy where he buys ETH when the price goes to 1500 USDC. He is willing to spend 5000 USDC for this. He would like to buy at this specific price only.
* **Buy Low, Sell High linked range trading orders** - Alice creates a strategy where she buys ETH as the price goes between 1500 and 1600 USDC. She is willing to spend 5000 USDC for this. She also wants to sell her acquired ETH when the price goes from 2000 to 2500 USDC.
* **Average In** - Jane creates an order to buy ETH between 1500 and 1600 USDC. She is willing to spend 5000 USDC for this.
* **Token Distribution** - Token project XYZ creates a strategy to sell 1 million units of its XYZ token in the price range of 0.50-2.00. The project would like to “back-load” the distribution process, such that half of the tokens are sold between 1.50-2.00. In total, the project wishes to receive a minimum of 1.37m in cash for its tokens.


In [1]:
from carbon import CarbonSimulatorUI
from IPython.display import HTML

## Limit Order - Sell ETH

### Initialize
We start by initializing the simulator with ETHUSDC as the default pair

In [2]:
Sim = CarbonSimulatorUI(pair="ETHUSDC")
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETHUSDC')

We remind ourselves the price convention for `ETHUSDC` (`ETH` or `USDC` has not semantic meaning here other than allowing to split the pair into its constituent tokens)

In [3]:
Sim.price_convention("ETHUSDC", "ETH"), Sim.price_convention("ETHUSDC", "USDC")

('USDC per ETH', 'USDC per ETH')

### Add Strategy

In [4]:
help(Sim.add_strategy)

Help on method add_strategy in module carbon.simulators.carbon_simulator:

add_strategy(tkn: str, amt_sell: Any, psell_start: Any, psell_end: Any, amt_buy: Any, pbuy_start: Any, pbuy_end: Any, pair: str = None) -> Dict[str, Any] method of carbon.simulators.carbon_simulator.CarbonSimulatorUI instance
    adds two linked orders (one buy, one sell; aka a "strategy")
    
    :tkn:           the token that is sold in the range psell_start, eg "ETH"*
    :amt_sell:      the amount of `tkn` that is available for sale in range psell_start/psell_end
    :psell_start:   start of the sell `tkn` range*, quoted in the price convention of `pair`
    :psell_end:     ditto end
    :amt_buy:       the amount of the other token that is available for selling against tkn in range pbuy_start
    :pbuy_start:    start of the of the buy `tkn` range*, quoted in the price convention of `pair`
    :pbuy_end:      ditto end
    :pair:          the token pair to which the strategy corresponds, eg "ETHUSD"
    
 

Thus when entering a strategy against the ETHUSDC pair prices are denoted in USDC

Here we set up a strategy where the AMM sells ETH for USDC at 2000 (USDC per ETH)
- The user is not interested in the buy side so sets zero liquidity and arbitrarily high buy price

In [5]:
Sim.add_strategy("ETH", 100, 2000, 2000, 0, 1000000, 1000000)["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,100.000000,100.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,0.000000,0.000000,USDC,1000000.000000,1000000.000000,USDC per ETH,0


We can access the global order list

In [6]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,100.000000,100.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,0.000000,0.000000,USDC,1000000.000000,1000000.000000,USDC per ETH,0


### Trade

In [7]:
help(Sim.amm_sells)

Help on method amm_sells in module carbon.simulators.carbon_simulator:

amm_sells(tkn: str, amt: Any, pair: str = None, execute: bool = True, inpair: bool = True, limit_price: Any = None) -> Dict[str, Any] method of carbon.simulators.carbon_simulator.CarbonSimulatorUI instance
    the AMM sells (and the trader buys) `amt` > 0 of `tkn`
    
    :tkn:           the token sold by the AMM and bought by the trader, eg "ETH"
    :amt:           the amount sold by the AMM and bought by the trader (must be positive)
    :pair:          the token pair to which the trade corresponds, eg "ETHUSD"
    :execute:       if True (default), the trade is executed; otherwise only routing is shown
    :inpair:        if True, only match within pair; if False (default), route through all available pairs
    :limit_price:   the limit price of the order (this price or better from point of view
                    of the trader, not the AMM!), quoted in convention of the pair



We can trade against the AMM by selling 10 ETH

In [8]:
Sim.amm_sells("ETH", 10)["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,0,0,A,AMM sells 10ETH buys 20000USDC,True,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


^^^ Where:
- `id` refers to a specific trade, 
- `subid` to one sub-route in the internal matching and, 
- `aggr` whether we refer to the individual sub-route or the aggregate of the sub-routes corresponding to the whole trade
- `exec` whether the trade was executed
- `limitfail` whether the order failed due to `limit_price` not being met
- `routeix` the indexes of the orders the trade was routed through - list for aggregate
- `nroutes` the number of orders the trade was routed through - 1 for sub-routes

We can access the global trade list

In [9]:
Sim.state()["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,0,0,A,AMM sells 10ETH buys 20000USDC,True,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


The full list includes all routes. To only look at the full trades use `query("aggr==True")`

In [10]:
Sim.state()["trades"].query("aggr==True")

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0,0,A,AMM sells 10ETH buys 20000USDC,True,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


We view the global trade list to see that indeed 10 ETH were removed from the ETH liquidity (y) and the proceeds accumulated to the USDC side

In [11]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,100.000000,90.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,20000.000000,20000.000000,USDC,1000000.000000,1000000.000000,USDC per ETH,0


## Limit Order - Buy ETH

Bob creates a strategy where he buys ETH when the price goes to 1500 USDC. He is willing to spend 5000 USDC for this. He would like to buy at this specific price only.

### Initialize

In [12]:
Sim = CarbonSimulatorUI(pair="ETHUSDC")
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETHUSDC')

### Add Strategy

Here we set up a strategy where the AMM buys ETH for USDC at 1500 (USDC per ETH)
- The user is not interested in the sell side so sets zero liquidity and arbitrarily high sell price

In [13]:
Sim.add_strategy('ETH', 0, 50000, 500000, 5000, 1500, 1500)['orders']

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,0.000000,0.000000,ETH,50000.000000,500000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,5000.000000,5000.000000,USDC,1500.000000,1500.000000,USDC per ETH,0


### Trade

In [14]:
help(Sim.amm_buys)

Help on method amm_buys in module carbon.simulators.carbon_simulator:

amm_buys(tkn: str, amt: Any, pair: str = None, execute: bool = True, inpair: bool = True, limit_price: Any = None) -> Dict[str, Any] method of carbon.simulators.carbon_simulator.CarbonSimulatorUI instance
    the AMM buys (and the trader sells) `amt` > 0 of `tkn`
    
    :tkn:           the token bought by the AMM and sold by the trader, eg "ETH"
    :amt:           the amount bought by the AMM and sold by the trader (must be positive)
    :pair:          the token pair to which the trade corresponds, eg "ETHUSD"
    :execute:       if True (default), the trade is executed; otherwise only routing is shown
    :inpair:        if True, only match within pair; if False (default), route through all available pairs
    :limit_price:   the limit price of the order (this price or better from point of view
                    of the trader, not the AMM!), quoted in convention of the pair



The AMM then buys ETH at an average price of 1500

In [15]:
Sim.amm_buys('ETH', 2)['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,None,3000.0,USDC,2.0,ETH,ETHUSDC,1,1,1500.0,USDC per ETH
0,0,0,A,AMM sells 3000USDC buys 2ETH,True,True,None,3000.0,USDC,2.0,ETH,ETHUSDC,[1],1,1500.0,USDC per ETH


The proceeds accumulate on the other side of the strategy

In [16]:
Sim.state()['orders']

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,2.000000,2.000000,ETH,50000.000000,500000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,5000.000000,2000.000000,USDC,1500.000000,1500.000000,USDC per ETH,0


## Average In

Jane creates an order to buy ETH between 1500 and 1600 USDC. She is willing to spend 5000 USDC for this.

### Initialize

In [17]:
Sim = CarbonSimulatorUI(pair="ETHUSDC")
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETHUSDC')

### Add Strategy

Here we set up a strategy where the AMM buys ETH for USDC at 1600-1500 (USDC per ETH)
- The user is not interested in the sell side so sets zero liquidity and arbitrarily high sell price

In [18]:
Sim.add_strategy('ETH', 0, 50000, 500000, 5000, 1600, 1500)['orders']

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,0.000000,0.000000,ETH,50000.000000,500000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,5000.000000,5000.000000,USDC,1600.000000,1500.000000,USDC per ETH,0


### Trade

The AMM then buys ETH in the associated range 1600-1500

In [19]:
Sim.amm_buys('ETH', 1)['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,None,1583.905411,USDC,1.0,ETH,ETHUSDC,1,1,1583.905411,USDC per ETH
0,0,0,A,AMM sells 1584USDC buys 1ETH,True,True,None,1583.905411,USDC,1.0,ETH,ETHUSDC,[1],1,1583.905411,USDC per ETH


And the USDC proceeds accumulate in the other side of the strategy

In [20]:
Sim.state()['orders']

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,1.000000,1.000000,ETH,50000.000000,500000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,5000.000000,3416.094589,USDC,1600.000000,1500.000000,USDC per ETH,0


We can then buy the remaining amount of ETH

In [21]:
Sim.amm_buys('ETH', 2.227486121)['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #1,False,True,None,3416.094588,USDC,2.227486,ETH,ETHUSDC,1,1,1533.609819,USDC per ETH
0,1,1,A,AMM sells 3416USDC buys 2ETH,True,True,None,3416.094588,USDC,2.227486,ETH,ETHUSDC,[1],1,1533.609819,USDC per ETH


Observe that the USDC reserve is empty

In [22]:
Sim.state()['orders']

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,3.227486,3.227486,ETH,50000.000000,500000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,5000.000000,0.000001,USDC,1600.000000,1500.000000,USDC per ETH,0


Inspect the trade summary

In [23]:
results = Sim.state()['trades'].query('aggr==True')
results

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0,0,A,AMM sells 1584USDC buys 1ETH,True,True,None,1583.905411,USDC,1.000000,ETH,ETHUSDC,[1],1,1583.905411,USDC per ETH
0,1,1,A,AMM sells 3416USDC buys 2ETH,True,True,None,3416.094588,USDC,2.227486,ETH,ETHUSDC,[1],1,1533.609819,USDC per ETH


And verify that the average price recieved is the same as the geometric average price for that order (sqrt(p_start*p_end))

In [24]:
results.sum()['amt1'] / results.sum()['amt2']

1549.193396656097

In [25]:
(1500*1600)**.5

1549.1933384829667

## Buy Low, Sell High - Range trading

Alice creates a strategy where she buys ETH as the price goes between 1500 and 1600 USDC. She is willing to spend 5000 USDC for this. She also wants to sell her acquired ETH when the price goes from 2000 to 2500 USDC.

### Initialize

In [26]:
Sim = CarbonSimulatorUI(pair="ETHUSDC")
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETHUSDC')

### Add Strategy

Here we set up a strategy where the AMM:
- Sells ETH for 2000-2500 and,
- Buys ETH for 1600-1500 (USDC per ETH)

In [27]:
Sim.add_strategy('ETH', 0, 2000, 2500, 5000, 1600, 1500)['orders']
# Sim.add_strategy('USDC', 5000, 1600, 1500, 0, 2000, 2500)['orders'] #(equivalent to)

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,0.000000,0.000000,ETH,2000.000000,2500.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,5000.000000,5000.000000,USDC,1600.000000,1500.000000,USDC per ETH,0


### Trade

The AMM buys ETH using the order reserves and we can see the average price in the range 1600-1500

In [28]:
Sim.amm_buys('ETH', 1)['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,None,1583.905411,USDC,1.0,ETH,ETHUSDC,1,1,1583.905411,USDC per ETH
0,0,0,A,AMM sells 1584USDC buys 1ETH,True,True,None,1583.905411,USDC,1.0,ETH,ETHUSDC,[1],1,1583.905411,USDC per ETH


Looking at the orders we can see that USDC has been deducted and ETH has been added to other side of the strategy

In [29]:
Sim.state()['orders']

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,1.000000,1.000000,ETH,2000.000000,2500.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,5000.000000,3416.094589,USDC,1600.000000,1500.000000,USDC per ETH,0


When the AMM then sells ETH the average price is now in the range 2000-2500

In [30]:
Sim.amm_sells('ETH', 1)['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #0,False,True,None,1.0,ETH,2236.067977,USDC,ETHUSDC,0,1,2236.067977,USDC per ETH
0,1,1,A,AMM sells 1ETH buys 2236USDC,True,True,None,1.0,ETH,2236.067977,USDC,ETHUSDC,[0],1,2236.067977,USDC per ETH


ETH has now been deducted and the USDC balance and the net profit (652.16) is available on the USDC side for purchasing more ETH

In [31]:
Sim.state()['orders']

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,1.000000,0.000000,ETH,2000.000000,2500.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,5652.162566,5652.162566,USDC,1600.000000,1500.000000,USDC per ETH,0


## Token Distribution

Token project XYZ creates a strategy to sell 1 million units of its XYZ token in the price range of 0.50-2.00. The project would like to “back-load” the distribution process, such that half of the tokens are sold between 1.50-2.00. In total, the project wishes to receive a minimum of 1.37m in cash for its tokens.

### Initialize
We start by initializing the simulator with XYZUSDC as the default pair

In [32]:
Sim = CarbonSimulatorUI(pair="XYZUSDC")
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='XYZUSDC')

### Add Strategies

- Strategy 1) Half of the tokens across the full range
    - The user is not interested in the buy side so sets zero liquidity and arbitrarily high buy price

In [33]:
Sim.add_strategy('XYZ', 500000, 0.5, 2, 0, 1000000, 1000000, pair='XYZUSDC')['orders']

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,XYZUSDC,XYZ,500000.000000,500000.000000,XYZ,0.500000,2.000000,USDC per XYZ,1
1,1,XYZUSDC,USDC,0.000000,0.000000,USDC,1000000.000000,1000000.000000,USDC per XYZ,0


- Strategy 2) Half of the tokens back-loaded at the high price
    - The user is not interested in the buy side so sets zero liquidity and arbitrarily high buy price

In [34]:
Sim.add_strategy('XYZ', 500000, 1.5, 2, 0, 1000000, 1000000, pair='XYZUSDC')['orders']

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
2,2,XYZUSDC,XYZ,500000.000000,500000.000000,XYZ,1.500000,2.000000,USDC per XYZ,3
3,3,XYZUSDC,USDC,0.000000,0.000000,USDC,1000000.000000,1000000.000000,USDC per XYZ,2


## Trade

Lets look at 5 trades to make up the entire amount:

While under 1.5 USDC per XYZ trades only go through the first order

In [35]:
Sim.amm_sells('XYZ', 200000)['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,200000.0,XYZ,125000.0,USDC,XYZUSDC,0,1,0.625,USDC per XYZ
0,0,0,A,AMM sells 200000XYZ buys 125000USDC,True,True,None,200000.0,XYZ,125000.0,USDC,XYZUSDC,[0],1,0.625,USDC per XYZ


We can see the average price increases per trade

In [36]:
Sim.amm_sells('XYZ', 200000)['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #0,False,True,None,200000.0,XYZ,208333.333333,USDC,XYZUSDC,0,1,1.041667,USDC per XYZ
0,1,1,A,AMM sells 200000XYZ buys 208333USDC,True,True,None,200000.0,XYZ,208333.333333,USDC,XYZUSDC,[0],1,1.041667,USDC per XYZ


The next 200000 tokens are sold for an average price > 1.5 so both orders are being accessed

In [37]:
Sim.amm_sells('XYZ', 200000)['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,2.0,2,0,route #0,False,True,None,46410.161514,XYZ,69862.435880,USDC,XYZUSDC,0,1,1.550677,USDC per XYZ
0,2.1,2,1,route #2,False,True,None,153589.838486,XYZ,240273.019201,USDC,XYZUSDC,2,1,1.550677,USDC per XYZ
0,2,2,A,AMM sells 200000XYZ buys 310135USDC,True,True,None,200000.000000,XYZ,310135.455081,USDC,XYZUSDC,"[0, 2]",2,1.550677,USDC per XYZ


In [38]:
Sim.amm_sells('XYZ', 200000)['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,3.0,3,0,route #0,False,True,None,26794.919243,XYZ,45940.188131,USDC,XYZUSDC,0,1,1.714511,USDC per XYZ
0,3.1,3,1,route #2,False,True,None,173205.080757,XYZ,296962.044301,USDC,XYZUSDC,2,1,1.714511,USDC per XYZ
0,3,3,A,AMM sells 200000XYZ buys 342902USDC,True,True,None,200000.000000,XYZ,342902.232433,USDC,XYZUSDC,"[0, 2]",2,1.714511,USDC per XYZ


In [39]:
Sim.amm_sells('XYZ', 200000)['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,4.0,4,0,route #0,False,True,None,26794.919243,XYZ,50864.042655,USDC,XYZUSDC,0,1,1.898272,USDC per XYZ
0,4.1,4,1,route #2,False,True,None,173205.080757,XYZ,328790.340282,USDC,XYZUSDC,2,1,1.898272,USDC per XYZ
0,4,4,A,AMM sells 200000XYZ buys 379654USDC,True,True,None,200000.000000,XYZ,379654.382938,USDC,XYZUSDC,"[0, 2]",2,1.898272,USDC per XYZ


Now we see that 5 trades have gone through and their sub-routing for each

In [40]:
results = Sim.state()['trades']
results

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,200000.000000,XYZ,125000.000000,USDC,XYZUSDC,0,1,0.625,USDC per XYZ
0,0,0,A,AMM sells 200000XYZ buys 125000USDC,True,True,None,200000.000000,XYZ,125000.000000,USDC,XYZUSDC,[0],1,0.625,USDC per XYZ
0,1.0,1,0,route #0,False,True,None,200000.000000,XYZ,208333.333333,USDC,XYZUSDC,0,1,1.041667,USDC per XYZ
0,1,1,A,AMM sells 200000XYZ buys 208333USDC,True,True,None,200000.000000,XYZ,208333.333333,USDC,XYZUSDC,[0],1,1.041667,USDC per XYZ
0,2.0,2,0,route #0,False,True,None,46410.161514,XYZ,69862.435880,USDC,XYZUSDC,0,1,1.550677,USDC per XYZ
0,2.1,2,1,route #2,False,True,None,153589.838486,XYZ,240273.019201,USDC,XYZUSDC,2,1,1.550677,USDC per XYZ
0,2,2,A,AMM sells 200000XYZ buys 310135USDC,True,True,None,200000.000000,XYZ,310135.455081,USDC,XYZUSDC,"[0, 2]",2,1.550677,USDC per XYZ
0,3.0,3,0,route #0,False,True,None,26794.919243,XYZ,45940.188131,USDC,XYZUSDC,0,1,1.714511,USDC per XYZ
0,3.1,3,1,route #2,False,True,None,173205.080757,XYZ,296962.044301,USDC,XYZUSDC,2,1,1.714511,USDC per XYZ
0,3,3,A,AMM sells 200000XYZ buys 342902USDC,True,True,None,200000.000000,XYZ,342902.232433,USDC,XYZUSDC,"[0, 2]",2,1.714511,USDC per XYZ


We can see the aggregates specifically

In [41]:
results.query("aggr==True")

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0,0,A,AMM sells 200000XYZ buys 125000USDC,True,True,None,200000.0,XYZ,125000.000000,USDC,XYZUSDC,[0],1,0.625,USDC per XYZ
0,1,1,A,AMM sells 200000XYZ buys 208333USDC,True,True,None,200000.0,XYZ,208333.333333,USDC,XYZUSDC,[0],1,1.041667,USDC per XYZ
0,2,2,A,AMM sells 200000XYZ buys 310135USDC,True,True,None,200000.0,XYZ,310135.455081,USDC,XYZUSDC,"[0, 2]",2,1.550677,USDC per XYZ
0,3,3,A,AMM sells 200000XYZ buys 342902USDC,True,True,None,200000.0,XYZ,342902.232433,USDC,XYZUSDC,"[0, 2]",2,1.714511,USDC per XYZ
0,4,4,A,AMM sells 200000XYZ buys 379654USDC,True,True,None,200000.0,XYZ,379654.382938,USDC,XYZUSDC,"[0, 2]",2,1.898272,USDC per XYZ


We can look back at the orders and see that the liquidity (y) is depleted for both XYZ sides

In [42]:
Sim.state()['orders']

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,XYZUSDC,XYZ,500000.000000,0.000000,XYZ,0.500000,2.000000,USDC per XYZ,1
1,1,XYZUSDC,USDC,499999.999999,499999.999999,USDC,1000000.000000,1000000.000000,USDC per XYZ,0
2,2,XYZUSDC,XYZ,500000.000000,0.000000,XYZ,1.500000,2.000000,USDC per XYZ,3
3,3,XYZUSDC,USDC,866025.403784,866025.403784,USDC,1000000.000000,1000000.000000,USDC per XYZ,2


In [43]:
Sim.state()['orders'].query("tkn_name=='XYZ'")[["y"]].sum()

y    0.000000
dtype: object

And see that the total amount of USDC obtained for the sale of 1M XYZ tokens was 1.336M

In [44]:
Sim.state()['orders'].query("tkn_name=='USDC'")[["y"]].sum()

y    1366025.403783
dtype: object